<a href="https://colab.research.google.com/github/Andy-yun-liang/TechnicalAssessment/blob/main/Seminar_Registration_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Library Imports

In [327]:
import pandas as pd
from datetime import datetime
from google.colab import files
import re
from io import BytesIO
from io import StringIO

Functions

In [307]:
def phoneNumberValidation(phone_str):


    if not isinstance(phone_str, str):
        return phone_str
    original_str = phone_str
    phone_str = phone_str.strip()


    if phone_str.startswith("1"):
        phone_str = phone_str[1:]


    cleaned_str = (
        phone_str.replace(".", "-")
        .replace("+1", "")
        .replace(" ", "-")
        .replace("(", "")
        .replace(")", "")
        .replace("/", "-")
    )


    cleaned_str = ''.join(char for char in cleaned_str if char.isdigit() or char == '-')


    cleaned_for_validation = cleaned_str.replace("-", "")


    if not cleaned_for_validation.isdigit() or len(cleaned_for_validation) != 10:
        return original_str


    return f"{cleaned_for_validation[:3]}-{cleaned_for_validation[3:6]}-{cleaned_for_validation[6:]}"




def emailValidation(email_str):

    original_email = email_str
    if not isinstance(email_str, str):
        return original_email


    email_str = email_str.replace(',', '.')
    email_str = email_str.lower()
    email_str = email_str.replace(' ', '')


    domains = ["hotmail", "gmail", "yahoo", "outlook"]
    for domain in domains:
        if domain in email_str and "@" not in email_str:
            email_str = email_str.replace(domain, "@" + domain)
            break


    pattern = r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"


    if re.match(pattern, email_str):
        return email_str
    else:
        return original_email



def upload_and_read_csv():

    uploaded = files.upload()


    file_name = next(iter(uploaded))


    if not file_name.endswith('.csv'):
        print("Please upload a CSV file!")
        return None


    data_csv = pd.read_csv(StringIO(uploaded[file_name].decode('utf-8')))

    return data_csv


def upload_and_read_excel():

    uploaded = files.upload()


    file_name = next(iter(uploaded))


    if not file_name.endswith('.xlsx'):
        print("Please upload an Excel (.xlsx) file!")
        return None


    data_excel = pd.read_excel(BytesIO(uploaded[file_name]))

    return data_excel


**First File, we are uploading is the CSV file we downloaded from SiteGrounds**

**Second File, we are uploading is the Seminar Registration Form from Microsoft**

In [308]:
db_data = upload_and_read_csv()

forms_data = upload_and_read_excel()

Saving weekly_seminar_tbl (3).csv to weekly_seminar_tbl (3) (9).csv


Saving Seminar Registration 精英理財講座 登記表(1-4416).xlsx to Seminar Registration 精英理財講座 登記表(1-4416) (9).xlsx


Data Manipulation on MSFT forms

In [309]:
forms_data.drop(columns=["ID","Start time","Email","Name"],inplace= True)

In [310]:
forms_data['Completion time'] = forms_data['Completion time'].dt.strftime('%d-%b-%y')

In [311]:
forms_data["Phone"] = forms_data["Mobile Phone Number 手提電話號碼"].apply(phoneNumberValidation)

In [312]:
forms_data["Email"] = forms_data["Email Address 電子郵箱"].apply(emailValidation)
forms_data['Email'] = forms_data['Email'].str.strip().str.lower()

In [313]:
#Dropping Email Address and Phone Number with Chinese Characters and also changing the name of "Completion Time" to "Seminar Registration Date" and "[Cantonese]" to "Seminar"

forms_data.drop(['Email Address 電子郵箱','Mobile Phone Number 手提電話號碼'],axis=1,inplace=True)
forms_data.rename(columns={'Completion time':'Seminar Registration Date','[Cantonese]':'Seminar'},inplace=True)

Data Manipulation on Elite Internal VIEW


In [314]:
db_data['phone_no'] = db_data['phone_no'].fillna(db_data['mobile_no'])

# Create a DataFrame where mobile_no is renamed to phone_no
mobile_df = db_data[['first_name', 'last_name', 'common_name', 'advisor_fname', 'advisor_lname', 'email', 'mobile_no', 'status_name']].copy()
mobile_df = mobile_df.rename(columns={"mobile_no": "phone_no"})

# Combine both DataFrames to have rows containing all phone_no and mobile_no data
combined_df = pd.concat([db_data[['first_name', 'last_name', 'common_name', 'advisor_fname', 'advisor_lname', 'email', 'phone_no', 'status_name']],
                         mobile_df],
                        ignore_index=True)

# Remove duplicates and sort
combined_df = combined_df.drop_duplicates().sort_values(['first_name', 'last_name', 'advisor_fname'])

# Remove rows where phone_no is NaN or empty
cleaned_db_data = combined_df.dropna(subset=['phone_no'])


In [315]:
dbData = cleaned_db_data.copy()
dbData['Assigned Advisor'] = dbData['advisor_fname'] + " "+ dbData['advisor_lname']
dbData['Client Name From Elite Internal'] = dbData["first_name"] + " " + dbData["last_name"]
dbData['Phone Number'] = dbData['phone_no']
dbData['Assigned Status'] = dbData['status_name']
dbData['Client Name From Elite Internal'] = dbData['Client Name From Elite Internal'].str.title().str.strip()
dbData['Email'] = dbData['email'].str.strip().str.lower()


In [316]:
columns_we_want = dbData[['Phone Number','Client Name From Elite Internal','Assigned Advisor','Assigned Status']]
columns_we_want = columns_we_want.drop_duplicates()

Joining the tables

In [317]:
left_join_tbl = pd.merge(forms_data, columns_we_want, left_on='Phone', right_on='Phone Number', how='left')
left_join_tbl.drop(['Phone Number'],axis = 1,inplace=True)

In [318]:
left_join_tbl['Assigned Advisor'] = left_join_tbl['Assigned Advisor'].fillna('New Client')
left_join_tbl['Assigned Status'] = left_join_tbl['Assigned Status'].fillna('New Client')
left_join_tbl['Client Name From Elite Internal'] = left_join_tbl['Client Name From Elite Internal'].fillna('New Client')
left_join_tbl['Source'] = left_join_tbl['How did you hear about this event? 您從何得知是次講座？']

In [319]:
left_join_tbl = left_join_tbl.drop_duplicates()

In [320]:
left_join_tbl.columns

Index(['Seminar Registration Date', 'Seminar', 'Seminar Format 講座形式',
       'First Name 名字', 'Last Name 姓氏',
       'Is this the first time you register for our seminar? 是否第一次參加我們的講座？',
       'Choose the one that describes you 您是以下哪一位', 'Your Occupation 您的職業',
       'What type of business do you own? 您的生意屬於哪種行業？',
       'Number of staff? 員工人數?', 'Previous year Corporate Tax 上年企業稅',
       'Principal Home Property Tax 自住物業地稅',
       'Do you own any rental properties? 您是否有收租物業？',
       'Type of rental properties that you own 您的收租物業屬於哪種？',
       'Your current RRSP / RRIF amount? 您當前的 RRSP / RRIF 金額？',
       'What are you hoping to learn from the seminar? 您希望從是次講座了解哪些資訊？',
       'How did you hear about this event? 您從何得知是次講座？',
       'Please enter the name of your referral:',
       'Who is your financial advisor at Elite? 哪一位是您的精英理財顧問？', 'Remarks 補充資料',
       'Phone', 'Email', 'Client Name From Elite Internal', 'Assigned Advisor',
       'Assigned Status', 'Source'],
      dtype

**If you want to remove a specific column look at the list above and copy and paste the above column in the box below with a , followed by quotation marks**

In [321]:
columns_to_be_removed=['Seminar Format 講座形式','What type of business do you own? 您的生意屬於哪種行業？','Number of staff? 員工人數?','Previous year Corporate Tax 上年企業稅','What are you hoping to learn from the seminar? 您希望從是次講座了解哪些資訊？','Please enter the name of your referral:']

In [322]:
left_join_tbl.drop(columns = columns_to_be_removed,inplace = True)

In [323]:
output_df = left_join_tbl[['Seminar Registration Date','Seminar', 'First Name 名字', 'Last Name 姓氏',
       'Email', 'Phone',
       'Is this the first time you register for our seminar? 是否第一次參加我們的講座？',
       'Choose the one that describes you 您是以下哪一位','Your Occupation 您的職業',
       'Principal Home Property Tax 自住物業地稅',
       'Type of rental properties that you own 您的收租物業屬於哪種？',
       'Your current RRSP / RRIF amount? 您當前的 RRSP / RRIF 金額？','Source',
       'Who is your financial advisor at Elite? 哪一位是您的精英理財顧問？', 'Remarks 補充資料',
       'Client Name From Elite Internal', 'Assigned Advisor',
       'Assigned Status']].copy()

output_df["Advisor to contact"] = output_df["Assigned Advisor"].apply(lambda x: x if "New Client" not in x else "Fill in Advisor Name")

**If you want a copy of the master list remove the # in the box below**

In [324]:
#output_df.to_csv("resulting.csv", index=False, encoding='utf-8-sig')

In [325]:
seminar_title = output_df["Seminar"].iloc[-1]
this_seminar = output_df[output_df["Seminar"]==seminar_title]

In [329]:
this_seminar.to_csv(f"current_seminar_generated_on_{datetime.now().date()}.csv", index=False, encoding='utf-8-sig')
files.download(f'current_seminar_generated_on_{datetime.now().date()}.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>